In [1]:
import sys
sys.path.insert(0,'/home/berhe/Desktop/Thesis_git/TLP_thesis/codes/')
import Eastenders_Lib as est
import Subtitle_truthvalue as stv
import evaluationMetrics as em
import pandas as pd
import os
from pyannote.metrics.diarization import DiarizationCoverage
import numpy as np

Subtitle files and Scene boundries files directory

In [2]:
scenes_dir='/home/berhe/Desktop/Thesis_git/TLP_thesis/Scenes/'
subtitle_dir='/home/berhe/Desktop/Thesis_git/TLP_thesis/subtitles/'

subtitleFiles=[]
for fl in os.listdir(subtitle_dir):
    if '.en.srt' in fl:
       subtitleFiles.append(fl)

scenesegementedFiles=[]
for fl in os.listdir(scenes_dir):
    if 'Episode' in fl:
       scenesegementedFiles.append(fl)
    
subtitleFiles.sort()
scenesegementedFiles.sort()

In [4]:
threads='/home/berhe/Desktop/Thesis_git/TLP_thesis/threadsShot/thread_output6_10_10.json'
shotSeq, shotBondry, _=est.preprocessingThreads(threads=threads)

In [5]:
scenesShotSequence=est.segmentation_speakers(shotSeq,7,4)

In [6]:
len(scenesShotSequence),len(shotSeq)

(23, 249)

In [7]:
sceneShotBoundry=[]
for i in scenesShotSequence:
    sceneShotBoundry.append(shotBondry[i])

start=sceneShotBoundry
start=[0.]+start
del start[-1]
len(start)

23

In [8]:
scenesegFile=scenes_dir+scenesegementedFiles[5]
subtFile=subtitle_dir+subtitleFiles[5]

In [9]:
_,_,sc_start,sc_end=stv.sceneTexts(scenesegFile,subtFile)

In [10]:
sceneShotBoundry[0:10],sc_start[0:3],(sc_end[0:3])

([50.36,
  71.08,
  76.76,
  90.2,
  435.56,
  653.36,
  745.24,
  1010.9200000000001,
  1081.88,
  1121.68],
 [0.0, 5800.0, 112900.0],
 [5800.0, 112900.0, 381400.0])

In [11]:
df=stv.get_text_time(subtFile)

In [13]:
len(df['start_seconds'])

652

In [15]:
df['textList'][1],df['end_second'][1],df['start_seconds'][1]

(u'Your pardon, Your Grace.', 141, 139)

In [16]:
scene_texts,scene_lbls,start,end=stv.sceneTexts(scenesegFile,subtFile)

In [17]:
len(scene_texts),len(scene_lbls),len(start),len(end)

(22, 22, 22, 22)

In [18]:
cl_sc=stv.split_into_sentences(scene_texts[2])

In [39]:
from nltk import sent_tokenize
cl_sc=sent_tokenize(scene_texts[10])

In [40]:
len(scene_texts[10]),len(cl_sc)

(1110, 24)

In [23]:
episodeTVM,episodeTVMB,l1=stv.episodTruthValueMan1(scenesegementedFile=scenesegFile,subtitleFile=subtFile)
episodesTVA,episodesTVAB,l2=stv.episodTruthValueShot1(scene_start=start,scene_end=sceneShotBoundry,subtitleFile=subtFile)

In [24]:
len(episodesTVA),len(episodeTVM),l1,l2

(652, 652, 652, 652)

In [25]:
episodesTVAB[25:40],episodesTVA[25:40]

([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [26]:
em.pk(episodeTVM,episodesTVA,k=15,boundary=1)

0.44200626959247646

In [27]:
em.pk(episodeTVMB,episodesTVAB,k=15,boundary=1)

0.0

In [28]:
em.windowdiff(episodeTVM,episodesTVA,k=13,boundary=1)

0.6025039123630673

In [29]:
em.windowdiff(episodeTVMB,episodesTVAB,k=15,boundary=1)

0.0

In [30]:
episodeTVM.count(1)+1

21

In [31]:
episodesTVA.count(1)+1

26

In [32]:
K=(len(episodeTVM)/(2*((episodeTVM.count(1)+1))))

In [33]:
K

15

In [34]:
kvalues=[]
cvalues=[]
windiffSHotA=[]
pkShotA=[]
shotRecA=[]
shotPrA=[]
shotRecB=[]
shotPrB=[]
sceneShot=[]
purityE=[]
NMIE=[]
purityEE=[]
NMIEE=[]
Coverage=[]
Coverage1=[]
for k in range(7, 11):
    for c in range(3,9):
        if c<k:
            scenesShotSequence=est.segmentation_speakers(shotSeq,k,c)
            sceneShotBoundry=[]
            for i in scenesShotSequence:
                sceneShotBoundry.append(shotBondry[i])

            start=sceneShotBoundry
            start=[0.]+start
            del start[-1]
            len(start)
            
            episodeTVM,episodeTVMB,l1=stv.episodTruthValueMan1(scenesegementedFile=scenesegFile,subtitleFile=subtFile)
            episodesTVA,episodesTVAB,l2=stv.episodTruthValueShot1(scene_start=start,scene_end=sceneShotBoundry,subtitleFile=subtFile)
            
            
            windiffSHot=round(em.windowdiff(episodeTVM,episodesTVA,k=K,boundary=1),2)
            pkShot=round(em.pk(episodeTVM,episodesTVA,k=K,boundary=1),2)
            
            episodesTVA=[int(i) for i in episodesTVA]
            episodeTVM=[int(i) for i in episodeTVM]
            episodesTVAB=[int(i) for i in episodesTVAB]
            episodeTVMB=[int(i) for i in episodeTVMB]
            episodesTVA=np.array(episodesTVA)
            episodeTVM=np.array(episodeTVM)
            episodesTVAB=np.array(episodesTVAB)
            episodeTVMB=np.array(episodeTVMB)
                
            shotRec=round(em.recall(episodeTVM,episodesTVA),3)
            shotPr=round(em.precision(episodeTVM,episodesTVA),3)
            shotRecb=round(em.recall(episodeTVMB,episodesTVAB,rType='micro'),3)
            shotPrb=round(em.precision(episodeTVMB,episodesTVAB,rType='micro'),3)
            puritye=round(em.purity_score(y_true=episodeTVMB,y_pred=episodesTVAB),3)
            nmie=round(em.nmi(episodeTVMB,episodesTVAB),3)
            purityee=round(em.purity_score(y_true=episodeTVM,y_pred=episodesTVA),3)
            nmiee=round(em.nmi(episodeTVM,episodesTVA),3)
            #coverage=DiarizationCoverage()
            dc=round(round(em.coverage(y_true=episodeTVM,y_pred=episodesTVA),3))
            dc1=round(round(em.purity_score(y_true=episodesTVA,y_pred=episodeTVM),3))
                
            
            kvalues.append(k)
            cvalues.append(c)
            windiffSHotA.append(windiffSHot)
            pkShotA.append(pkShot)
            shotRecA.append(shotRec)
            shotRecB.append(shotRecb)
            purityE.append(puritye)
            NMIE.append(nmie)
            purityEE.append(purityee)
            NMIEE.append(nmiee)
            shotPrA.append(shotPr)
            shotPrB.append(shotPrb)
            sceneShot.append(len(scenesShotSequence))
            Coverage.append(dc)
            Coverage1.append(dc1)
        else:
            break

In [35]:
resulDF=pd.DataFrame()
resulDF['K']=kvalues
resulDF['C']=cvalues
resulDF['wdSH']=windiffSHotA
resulDF['pkSh']=pkShotA
#resulDF['rSh']=recallShot
resulDF['shotRec_01']=shotRecB
resulDF['shotPr_01']=shotPrB
resulDF['shotRec']=shotRecA
resulDF['shotPr']=shotPrA
resulDF['sceneShot']=sceneShot
resulDF['purity_01']=purityEE
resulDF['NMI_01']=NMIEE
resulDF['purity']=purityE
resulDF['NMI']=NMIE
resulDF['Coverage']=Coverage
resulDF['Coverage1']=Coverage1

In [36]:
resulDF

,K,C,wdSH,pkSh,shotRec_01,shotPr_01,shotRec,shotPr,sceneShot,purity_01,NMI_01,purity,NMI,Coverage,Coverage1
0,7,3,0.76,0.40,0.028,0.028,0.20,0.121,34,0.969,0.027,0.080,0.800,1.0,1.0
1,7,4,0.66,0.44,0.043,0.043,0.15,0.120,23,0.969,0.022,0.126,0.783,1.0,1.0
2,7,5,1.42,0.45,0.017,0.017,0.20,0.069,17,0.969,0.009,0.137,0.732,1.0,1.0
3,7,6,1.69,0.51,0.012,0.012,0.20,0.062,13,0.969,0.007,0.084,0.682,1.0,1.0
4,8,3,0.76,0.40,0.028,0.028,0.20,0.121,34,0.969,0.027,0.080,0.800,1.0,1.0
5,8,4,0.66,0.44,0.043,0.043,0.15,0.120,23,0.969,0.022,0.126,0.783,1.0,1.0
6,8,5,1.42,0.45,0.017,0.017,0.20,0.069,17,0.969,0.009,0.137,0.732,1.0,1.0
7,8,6,1.69,0.51,0.012,0.012,0.20,0.062,13,0.969,0.007,0.084,0.682,1.0,1.0
8,8,7,1.64,0.47,0.098,0.098,0.25,0.078,12,0.969,0.015,0.137,0.711,1.0,1.0
9,9,3,0.76,0.40,0.028,0.028,0.20,0.121,34,0.969,0.027,0.080,0.800,1.0,1.0


In [38]:
resulDF.loc[resulDF['purity'].idxmax()]

K              7.000
C              4.000
wdSH           0.600
pkSh           0.390
shotRec_01     0.037
shotPr_01      0.037
shotRec        0.136
shotPr         0.136
sceneShot     23.000
purity_01      0.675
NMI_01         0.794
purity         0.675
NMI            0.794
Coverage       1.000
Coverage1      1.000
Name: 1, dtype: float64

In [37]:
resulDF.loc[resulDF['pkSh'].idxmin()]

K             9.000
C             8.000
wdSH          0.450
pkSh          0.350
shotRec_01    0.006
shotPr_01     0.006
shotRec       0.053
shotPr        0.111
sceneShot     9.000
purity_01     0.520
NMI_01        0.723
purity        0.520
NMI           0.723
Coverage      1.000
Coverage1     1.000
Name: 14, dtype: float64

In [93]:
resulDF.max(axis=None, skipna=None, level=None, numeric_only=None)

K             10.000
C              8.000
wdSH           0.550
pkSh           0.370
shotRec_01     0.100
shotPr_01      0.007
shotRec        0.100
shotPr         0.068
sceneShot     23.000
purity_01      0.703
NMI_01         0.787
purity         0.703
NMI            0.787
dtype: float64